# MiSaTo-Dataset: a tutorial

In this notebook, we will show how our QM and MD dataset are stored in h5 files. We also show how the data can be loaded so that it can be used by a deep learning model.

We start by importing the useful packages and set up the paths of the files

In [ ]:
!git clone https://github.com/sab148/MiSaTo-dataset.git

Cloning into 'MiSaTo-dataset'...
remote: Enumerating objects: 799, done.
remote: Counting objects: 100% (743/743), done.
remote: Compressing objects: 100% (371/371), done.
remote: Total 799 (delta 375), reused 676 (delta 350), pack-reused 56
Receiving objects: 100% (799/799), 173.99 MiB | 13.66 MiB/s, done.
Resolving deltas: 100% (375/375), done.
Updating files: 100% (91/91), done.


In [ ]:
!pip install torch-geometric
!pip install lightning
!pip install torch-sparse
!pip install torch-scatter
# NOTE: PyTorch pinned because https://github.com/conda-forge/openmm-torch-feedstock/issues/20
#!mamba install -q -c conda-forge \
 #              openmm-torch nnpops torchani openmmtools \
  #             pytorch=1.11 \
   #            &> /dev/null # Comment this line to see a log"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910460 sha256=13dd24eb3198addd3416e347eab4d2c952a82372e37db3c71a42a14e68e73b3b
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.7/596.7 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.1 MB/s eta 0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd MiSaTo-dataset/src/data

/content/MiSaTo-dataset/src/data


In [ ]:
%cd data

[Errno 2] No such file or directory: 'data'
/content/MiSaTo-dataset/src/data


In [ ]:
%cd components

/content/MiSaTo-dataset/src/data/components


In [ ]:
import sys
import os
sys.path.insert(0,os.path.join(os.path.abspath('').split('MiSaTo-dataset')[0],'MiSaTo-dataset/src/data/components/'))

import h5py
import numpy as np

import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader
from datasets import MolDataset, ProtDataset
from transformQM import GNNTransformQM
from transformMD import GNNTransformMD


In [ ]:
%cd /content/MiSaTo-dataset/src/data
%pwd

/content/MiSaTo-dataset/src/data


'/content/MiSaTo-dataset/src/data'

In [ ]:
from qm_datamodule import QMDataModule
from md_datamodule import MDDataModule
from processing import preprocessing_db

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'transforms.RandomTranslate' is deprecated, use 'transforms.RandomJitter' instead
  warnings.warn(out)


In [ ]:
#qmh5_file = "/content/MiSaTo_dataset/data/QM/h5_files/tiny_qm.hdf5"
qmh5_file = "/content/drive/MyDrive/MiSaTo/QM.hdf5"
norm_file = "/content/MiSaTo_dataset/data/QM/h5_files/qm_norm.hdf5"
norm_txtfile = "/content/MiSaTo_dataset/data/QM/splits/train_norm.txt"

 **Trying New Data Sets**

In [ ]:
import pandas as pd
import numpy as np




In [ ]:
newData1 = "/content/drive/MyDrive/pdbIndex/INDEX_general_PL.csv"
fd=pd.read_csv(newData1)
print("The dataframe is:")
print(fd)

The dataframe is:
      Column1  Column2  Column3 Column4  Column5  Column6    Column7  Column8  \
0        2tpi      NaN     2.10     NaN   1982.0      NaN    Kd=49uM      NaN   
1        5tln      NaN     2.30     NaN   1982.0      NaN  Ki=0.43uM      NaN   
2        4tln      NaN     2.30     NaN   1982.0      NaN   Ki=190uM      NaN   
3        4cts      NaN     2.90     NaN   1984.0      NaN    Kd<10uM      NaN   
4        6rsa      NaN      NaN     NMR      NaN   1986.0        NaN  Ki=40uM   
...       ...      ...      ...     ...      ...      ...        ...      ...   
19438    6gxe      NaN     1.30     NaN   2019.0      NaN     Ki=9nM      NaN   
19439    6r0u      NaN     1.70     NaN   2019.0      NaN    Ki>40uM      NaN   
19440    6r0s      NaN     1.55     NaN   2019.0      NaN    Ki>40uM      NaN   
19441    6r12      NaN     1.74     NaN   2019.0      NaN    Ki>40uM      NaN   
19442    6r1x      NaN     1.80     NaN   2019.0      NaN    Ki>40uM      NaN   

         

## H5 files presentations

We read the QM H5 file and H5 file used to normalize the target values.

In [ ]:
qm_H5File = h5py.File(qmh5_file)
qm_normFile = h5py.File(norm_file)

The ligands can be accessed using the pdb-id. Bellow we show the first ten molecules of the file.

In [ ]:
qm_H5File.keys()

<KeysViewHDF5 ['10GS', '11GS', '13GS', '16PK', '184L', '185L', '186L', '187L', '188L', '1A07', '1A08', '1A09', '1A0Q', '1A0TA', '1A0TB', '1A1B', '1A1C', '1A1E', '1A28', '1A2C', '1A30', '1A37', '1A42', '1A46', '1A4G', '1A4H', '1A4K', '1A4M', '1A4Q', '1A4R', '1A4W', '1A50', '1A52', '1A5G', '1A5H', '1A5V', '1A61', '1A69', '1A7C', '1A7T', '1A7X', '1A85', '1A86', '1A8I', '1A8T', '1A94', '1A99', '1A9M', '1A9Q', '1A9U', '1AAQ', '1ABF', '1ABT', '1ACJ', '1AD8', '1ADD', '1ADL', '1ADO', '1AF2', '1AF6', '1AFK', '1AFL', '1AG9', '1AGM', '1AGW', '1AHT', '1AHX', '1AHY', '1AI4', '1AI5', '1AI6', '1AI7', '1AID', '1AJ6', '1AJ7', '1AJN', '1AJP', '1AJQ', '1AJV', '1AJX', '1AKQ', '1AKR', '1AKT', '1AKU', '1AKV', '1AKW', '1AL7', '1AL8', '1ALW', '1AMK', '1AMN', '1AMW', '1ANF', '1AO0', '1AO8', '1APB', '1APV', '1APW', '1AQ1', '1AQ7', '1AQC', '1AQI', '1AQJ', '1AT5', '1AT6', '1ATL', '1ATR', '1AU0', '1AU2', '1AUJ', '1AVD', '1AVN', '1AVP', '1AW1', '1AWF', '1AWH', '1AWI', '1AX0', '1AX1', '1AX2', '1AXR', '1AXS', '1AXZ',

In [ ]:
qmList=qm_H5File.keys()
list1=[]
for i in qmList:
  list1.append(i)

In [ ]:
specific_columns = fd["Column1"].tolist()
j = 0
list2 = []
for i in specific_columns:
    list2.append(i.upper())
for i in list1:
    if i in list2:
           j += 1
print(list1)
print(list2)
print("no.of common molecules",j)
print("no of molecules in misato",len(list1))
print("no of molecules in pdbbind",len(list2))

['10GS', '11GS', '13GS', '16PK', '184L', '185L', '186L', '187L', '188L', '1A07', '1A08', '1A09', '1A0Q', '1A0TA', '1A0TB', '1A1B', '1A1C', '1A1E', '1A28', '1A2C', '1A30', '1A37', '1A42', '1A46', '1A4G', '1A4H', '1A4K', '1A4M', '1A4Q', '1A4R', '1A4W', '1A50', '1A52', '1A5G', '1A5H', '1A5V', '1A61', '1A69', '1A7C', '1A7T', '1A7X', '1A85', '1A86', '1A8I', '1A8T', '1A94', '1A99', '1A9M', '1A9Q', '1A9U', '1AAQ', '1ABF', '1ABT', '1ACJ', '1AD8', '1ADD', '1ADL', '1ADO', '1AF2', '1AF6', '1AFK', '1AFL', '1AG9', '1AGM', '1AGW', '1AHT', '1AHX', '1AHY', '1AI4', '1AI5', '1AI6', '1AI7', '1AID', '1AJ6', '1AJ7', '1AJN', '1AJP', '1AJQ', '1AJV', '1AJX', '1AKQ', '1AKR', '1AKT', '1AKU', '1AKV', '1AKW', '1AL7', '1AL8', '1ALW', '1AMK', '1AMN', '1AMW', '1ANF', '1AO0', '1AO8', '1APB', '1APV', '1APW', '1AQ1', '1AQ7', '1AQC', '1AQI', '1AQJ', '1AT5', '1AT6', '1ATL', '1ATR', '1AU0', '1AU2', '1AUJ', '1AVD', '1AVN', '1AVP', '1AW1', '1AWF', '1AWH', '1AWI', '1AX0', '1AX1', '1AX2', '1AXR', '1AXS', '1AXZ', '1AYU', '1AYV

In [ ]:
specific_columns2 = fd[["Column1","Column9"]].values.tolist()
print(specific_columns2)

[['2tpi', 'Kd=49uM'], ['5tln', 'Ki=0.43uM'], ['4tln', 'Ki=190uM'], ['4cts', 'Kd<10uM'], ['6rsa', 'Ki=40uM'], ['1rnt', 'Kd=6.5uM'], ['6cha', 'Ki=40uM'], ['4ts1', 'Kd=11.6uM'], ['4tmn', 'Ki=0.068nM'], ['2tmn', 'Ki=1.3uM'], ['1tlp', 'Ki=28nM'], ['1tmn', 'Ki=50nM'], ['5tmn', 'Ki=9.1nM'], ['4fab', 'Kd=8.8nM'], ['1p01', 'Ki=0.35nM'], ['3at1', 'Ki=0.66mM'], ['1p05', 'Ki=1100nM'], ['1p10', 'Ki=200nM'], ['6gch', 'Ki=20uM'], ['7gch', 'Ki=2uM'], ['1p04', 'Ki=40nM'], ['1p06', 'Ki=540nM'], ['1p03', 'Ki=6.4nM'], ['1p02', 'Ki=67nM'], ['8abp', 'Kd=0.010uM'], ['1rbp', 'Kd=0.19uM'], ['7abp', 'Kd=0.35uM'], ['6abp', 'Kd=0.44uM'], ['1fkf', 'Kd=0.4nM'], ['9icd', 'Kd=125uM'], ['06-Apr', 'Ki<=17nM'], ['4er1', 'Ki=0.242uM'], ['5er2', 'Ki=0.27uM'], ['4er2', 'Ki=0.5nM'], ['2ypi', 'Ki=15uM'], ['4er4', 'Ki=160nM'], ['05-Apr', 'Ki=17nM'], ['3er5', 'Ki=1nM'], ['04-Apr', 'Ki=200nM'], ['5hvp', 'Ki=20nM'], ['6cpa', 'Ki=3pM'], ['2er9', 'Ki=40nM'], ['2er0', 'Ki=420nM'], ['4sga', 'Ki=50nM'], ['2er6', 'Ki=60nM'], ['3er3', 

The following properties are available for each atom:

In [ ]:
qm_H5File["10GS"]["atom_properties"]["atom_properties_names"][()]

array([b'x', b'y', b'z', b'hybridisation', b'group', b'gfn2_charge',
       b'gfn2_charge_(wet_octanol)', b'gfn2_charge_(water)',
       b'AM1_charge', b'AM1_CM1_charge', b'AM1_CM2_charge',
       b'AM1_CM3_charge', b'PM6_charge', b'gfn2_polarisation',
       b'gfn2_polarisation_(wet_octanol)', b'gfn2_polarisation_(water)',
       b'gfn2_charge_electrophilicity',
       b'gfn2_charge_electrophilicity_softness',
       b'gfn2_charge_nucleophilicity_softness',
       b'gfn2_charge_nucleophilicity', b'gfn2_charge_radical',
       b'gfn2_charge_radical_softness', b'gfn2_orbital_electrophilicity',
       b'gfn2_orbital_electrophilicity_softness',
       b'gfn2_orbital_nucleophilicity_softness',
       b'gfn2_orbital_nucleophilicity', b'gfn2_orbital_radical',
       b'gfn2_orbital_radical_softness'], dtype='|S38')

You can access the values for each of the properties using the respective index. For example the coordinates are given in the first 3 entries:

In [ ]:
xyz = qm_H5File["10GS"]["atom_properties"]["atom_properties_values"][:, 0:3]

We also provide several molecular properties that can be accessed directly using the respective key.

In [ ]:
qm_H5File["10GS"]["mol_properties"].keys()

<KeysViewHDF5 ['Electron_Affinity', 'Electronegativity', 'Hardness', 'Ionization_Potential', 'Koopman', 'molecular_weight', 'total_charge']>

Target values can be accessed by specifiying into bracket the molecule name, then mol_properties and finally the name of the target value that we want to access:

In [ ]:
qm_H5File["10GS"]["mol_properties"]["Electron_Affinity"][()]

6.0974

We can access to the mean and standard-deviation of each target value over all structures by specifiying it into bracket.
We first specify the set, then the target value and finally either mean or std.

In [ ]:
qm_normFile.keys()

<KeysViewHDF5 ['Electron_Affinity', 'Electronegativity', 'Hardness', 'Ionization_Potential']>

In [ ]:
print(qm_normFile["Electron_Affinity"]["mean"][()])
print(qm_normFile["Electron_Affinity"]["std"][()])

6.33265
18.636927


## Datasets and dataloaders

### PyTorch

The QM and MD datasets are warped into a PyTorch Dataset class under the name MolDataset and ProtDataset, respectively.
The parameters taken by the two classes as well as their types can be found as follow.

In [ ]:
help(MolDataset)

In [ ]:
help(ProtDataset)

In [ ]:
%cd /content/MiSaTo_dataset/data/QM/splits/
%pwd

/content/MiSaTo_dataset/data/QM/splits


'/content/MiSaTo_dataset/data/QM/splits'

We can load the data by instanciating MolDataset and providing the QM H5 file, the text file that indicates the molecule used for training and the norm file used to normalize the target values.

The MolDataset class without any transform return a dictionary that contain the elements and their coordinates. We use GNNTransformQM class to transform our data to a graph that can be used by a GNN. The parameter post_transform is another transformation used to perform data augmentation.

In [ ]:
train = "train_tinyQM.txt"

transform = T.RandomTranslate(0.25)
batch_size = 128
num_workers = 48

data_train = MolDataset(qmh5_file, train, target_norm_file=norm_file, transform=GNNTransformQM(), post_transform=transform)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'transforms.RandomTranslate' is deprecated, use 'transforms.RandomJitter' instead
  warnings.warn(out)


Finally, we can load our data using the PyTorch DataLoader.

In [ ]:
train_loader = DataLoader(data_train, batch_size, shuffle=True, num_workers=0)

for idx, val in enumerate(train_loader):
    print(val)
    break

DataBatch(x=[397, 25], edge_index=[2, 6548], edge_attr=[6548, 1], y=[20], pos=[397, 3], id=[10], batch=[397], ptr=[11])


### PyTorch lightning

The QMDataModule is a class inherated from LightningDataModule that instanciate the MolDataset for training, validation and test set and returns a dataloader for each set.

We start by instanciation of the QMDataModule

In [ ]:
files_root =  "/content/MiSaTo_dataset/data/QM"

qmh5file = "h5_files/tiny_qm.hdf5"

tr = "splits/train_tinyQM.txt"
v = "splits/val_tinyQM.txt"
te = "splits/test_tinyQM.txt"

qmdata = QMDataModule(files_root, h5file=qmh5file, train=tr, val=v, test=te, num_workers=0)

Then, we call the setup function to instanciate the MolDataset for training, validation and test set

In [ ]:
%pwd


'/content/MiSaTo_dataset/data/QM/splits'

In [ ]:
qmdata.setup()

Finally, we can return a dataloader for each set.

In [ ]:
train_loader = qmdata.train_dataloader()

for idx, val in enumerate(train_loader):
    print(val)
    break


DataBatch(x=[397, 25], edge_index=[2, 6548], edge_attr=[6548, 1], y=[20], pos=[397, 3], id=[10], batch=[397], ptr=[11])


# MD dataset

We generated a tiny h5 file that can be inspected right away. We do this for the structure with pdb-id 10GS.

In [ ]:
mdh5_file_tiny = '/content/drive/MyDrive/MiSaTo/MD.hdf5'
md_H5File_tiny = h5py.File(mdh5_file_tiny)

In [ ]:
md_H5File_tiny['10GS'][frames_bSASA].keys()
#md_H5File_tiny.keys()

NameError: ignored

The beginning of the name of each property indicates the respective shape:
- atoms_ have a entry for each atom of the structure
- frames_ have an entry for each of the 100 frames
- molecules_ has an entry for each molecule, including the ligand
- trajectory_coordinates_ has an entry of each atom and each frame

In [ ]:
[(key, np.shape(md_H5File_tiny['10GS'][key])) for key in md_H5File_tiny['10GS'].keys()]

[('atoms_element', (6593,)),
 ('atoms_number', (6593,)),
 ('atoms_residue', (6593,)),
 ('atoms_type', (6593,)),
 ('frames_bSASA', (100,)),
 ('frames_distance', (100,)),
 ('frames_interaction_energy', (100,)),
 ('frames_rmsd_ligand', (100,)),
 ('molecules_begin_atom_index', (3,)),
 ('trajectory_coordinates', (100, 6593, 3))]

To run models for the MD dataset you will most likely need to preprocess the h5 file based on your model. We provide a preprocessing script (see data/processing/preprocessing_db.py) that can filter out the atom types that you are not interested in (e.g. H-atoms) or calculate values of interest based on your models.
Here, we will show how to use the script to calculate the adaptability values on the dataset and stripping the H-atoms.  
In this notebook we define a new Args class, if you use the script in the terminal just provide these values as input parameters in the command line.

In [ ]:
class Args:
    # input file
    datasetIn = "/content/MiSaTo_dataset/data/MD/h5_files/tiny_md.hdf5"
    # Feature that should be stripped, e.g. atoms_element or atoms_type
    strip_feature = "atoms_element"
    # Value to strip, e.g. if strip_freature= atoms_element; 1 for H.
    strip_value = 1
    # Start index of structures
    begin = 0
    # End index of structures
    end = 20
    # We calculate the adaptability for each atom.
    # Default behaviour will also strip H atoms, if no stripping should be perfomed set strip_value to -1.
    Adaptability = True
    # If set to True this will create a new feature that combines one entry for each protein AA but all ligand entries;
    # e.g. for only ca set strip_feature = atoms_type and strip_value = 14
    Pres_Lat = False
    # We strip the complex by given distance (in Angstrom) from COG of molecule,
    # use e.g. 15.0. If default value is given (0.0) no pocket stripping will be applied.
    Pocket = 0.0
    # output file name and location
    datasetOut = "/content/MiSaTo_dataset/data/MD/h5_files/tiny_md_out.hdf5"


args = Args()

preprocessing_db.main(args)

Removing existing output file...
10GS 1
Stripping  atoms_element 1  and calculating adaptability for the atoms that were not stripped.
11GS 2
13GS 3
16PK 4
184L 5
185L 6
186L 7
187L 8
188L 9
1A07 10
1A08 11
1A09 12
1A0Q 13
1A1B 14
1A1C 15
1A1E 16
1A28 17
1A2C 18
1A30 19
1A3E 20


The same steps used for QM can be used to load the MD dataset. We start by loading the generated h5 file.

In [ ]:
files_root =  ""

mdh5_file = 'MiSaTo_dataset/data/MD/h5_files/tiny_md_out.hdf5'

train_idx = "MiSaTo_dataset/data/MD/splits/train_tinyMD.txt"
val_idx = "MiSaTo_dataset/data/MD/splits/val_tinyMD.txt"
test_idx = "MiSaTo_dataset/data/MD/splits/test_tinyMD.txt"

md_H5File = h5py.File(mdh5_file)

FileNotFoundError: ignored

During preprocessing the H-atoms were stripped (see the change in atoms_ shape) and a new feature, the adaptability was calculated for each atom.

In [ ]:
[(key, np.shape(md_H5File['10GS'][key])) for key in md_H5File['10GS'].keys()]

In [ ]:
# Atom's coordinates from the first frame
xyz = md_H5File['10GS']['trajectory_coordinates'][0, :, :]

We can now initiate the dataloader.

In [ ]:
train_dataset = ProtDataset(mdh5_file, idx_file=train_idx, transform=GNNTransformMD(), post_transform=T.RandomTranslate(0.05))

train_loader = DataLoader(train_dataset, batch_size=16, num_workers=16)

In [ ]:
for idx, val in enumerate(train_loader):
    print(val)
    break

In [ ]:
mddata = MDDataModule(files_root, h5file=mdh5_file, train=train_idx, val=val_idx, test=test_idx, num_workers=0)

In [ ]:
mddata.setup()

In [ ]:
train_loader = mddata.train_dataloader()

for idx, val in enumerate(train_loader):
    print(val)
    break